# Interest Rate Swap

An Interest Rate Swap is a financial derivative instrument in which two parties agree to exchange interest rate cash flows based on a notional amount from a fixed rate to a floating rate or from one floating rate to another floating rate.

In [12]:
import QuantLib as ql
calculation_date = ql.Date(1, 7, 2020)
ql.Settings.instance().evaluationDate = calculation_date

In [13]:
# For simplicity, using flat curves for discounting and Libor 3M
# Constructing discount curve and libor curve

risk_free_rate = 0.02
libor_rate = 0.03
day_count = ql.Actual365Fixed()

discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
# YieldTermStructureHandle: gives the discount factor for a particular date
# FlatForward: constructing a rate curve that will lead to flat forward rates

libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, libor_rate, day_count))


libor3M_index = ql.USDLibor(ql.Period(3, ql.Months), libor_curve)

In [14]:
# Specify the fixed rate leg and floating rate leg

calendar = ql.UnitedStates()
settle_date = calendar.advance(calculation_date, 5, ql.Days)
maturity_date = calendar.advance(settle_date, 10, ql.Years)

fixed_leg_tenor = ql.Period(6, ql.Months)
fixed_schedule = ql.Schedule(settle_date, maturity_date, fixed_leg_tenor, calendar, ql.ModifiedFollowing, 
                             ql.ModifiedFollowing, ql.DateGeneration.Forward, False)

float_leg_tenor = ql.Period(3, ql.Months)
float_schedule = ql.Schedule (settle_date, maturity_date, float_leg_tenor, calendar, ql.ModifiedFollowing, 
                              ql.ModifiedFollowing, ql.DateGeneration.Forward, False)


In [15]:
# Vanilla Swap

notional = 1000000
fixed_rate = 0.025
fixed_leg_daycount = ql.Actual360()
float_spread = 0.004
float_leg_daycount = ql.Actual360()

ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, 
               fixed_rate, fixed_leg_daycount, float_schedule,
               libor3M_index, float_spread, float_leg_daycount )

In [16]:
# Evaluate swap using  DiscountingSwapEngine: Basic discounting bond engine using a yield term structure for pricing

swap_engine = ql.DiscountingSwapEngine(discount_curve)
ir_swap.setPricingEngine(swap_engine)

# Results

In [17]:
# Fixed Leg Cashflows
for i, cf in enumerate(ir_swap.leg(0)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    January 11th, 2021    12916.67
 2    July 9th, 2021        12430.56
 3    January 10th, 2022    12847.22
 4    July 11th, 2022       12638.89
 5    January 9th, 2023     12638.89
 6    July 10th, 2023       12638.89
 7    January 9th, 2024     12708.33
 8    July 9th, 2024        12638.89
 9    January 9th, 2025     12777.78
10    July 9th, 2025        12569.44
11    January 9th, 2026     12777.78
12    July 9th, 2026        12569.44
13    January 11th, 2027    12916.67
14    July 9th, 2027        12430.56
15    January 10th, 2028    12847.22
16    July 10th, 2028       12638.89
17    January 9th, 2029     12708.33
18    July 9th, 2029        12569.44
19    January 9th, 2030     12777.78
20    July 9th, 2030        12569.44


In [18]:
# Floating Leg Cashflows
for i, cf in enumerate(ir_swap.leg(1)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    October 9th, 2020      8612.53
 2    January 11th, 2021     8800.39
 3    April 9th, 2021        8236.87
 4    July 9th, 2021         8518.60
 5    October 12th, 2021     8894.34
 6    January 10th, 2022     8424.69
 7    April 11th, 2022       8518.60
 8    July 11th, 2022        8518.60
 9    October 11th, 2022     8612.53
10    January 9th, 2023      8424.69
11    April 10th, 2023       8518.91
12    July 10th, 2023        8518.30
13    October 10th, 2023     8612.53
14    January 9th, 2024      8518.60
15    April 9th, 2024        8518.60
16    July 9th, 2024         8518.60
17    October 9th, 2024      8612.53
18    January 9th, 2025      8612.53
19    April 9th, 2025        8424.69
20    July 9th, 2025         8518.60
21    October 9th, 2025      8612.53
22    January 9th, 2026      8612.53
23    April 9th, 2026        8424.69
24    July 9th, 2026         8518.60
25    October 9th, 2026      8612.53
26    January 11th, 2027     8800.39
27    April 9th, 2027        8236.87
2